In [0]:
import pyaudio
import math
from numpy import linspace,sin,pi,int16,rint,sign,arcsin,tan,arctan,cos,append,multiply,add,subtract,divide,repeat,random
import time
import pandas as pd
from enum import Enum
from scipy.io import wavfile

In [0]:
%matplotlib inline

In [0]:
for i in range(p.get_device_count()):
  dev = p.get_device_info_by_index(i)
  print((i,dev['name'],dev['maxInputChannels']))

In [0]:
from google.colab import files

with open('test.txt', 'w') as f:
  f.write('hello')
  
files.download('test.txt')

In [17]:
RATE=44100
p = pyaudio.PyAudio()
stream = p.open(output=True, channels=2, rate=RATE, format=pyaudio.paInt16, output_device_index=1)

OSError: ignored

In [0]:
class Note(Enum):
    C3 = 131
    D3 = 147
    E3 = 165
    F3 = 175
    G3 = 196
    A4 = 220
    B4 = 247
    C4 = 262

In [0]:
def inspect_sound(sound_data):
    df = pd.DataFrame(sound_data)
    print("shape: ", df.shape)
    print("head: ")
    df.head(101).plot()
    stream.write(df.values)

In [0]:
def wave(frequency=Note.C3.value, length=1, amplitude=5000, phase=0, sample_rate=RATE, function=None, function_args=None):
    time = linspace(0,length,length*sample_rate)
    wavelength = 1/frequency
    data = function(time, wavelength, amplitude, phase, function_args)
    return data.astype(int16) # two byte integers

In [0]:
def sine_function(time, wavelength, amplitude, phase, function_args=None):
    return amplitude * sin((2*pi*time - phase)/wavelength)

In [0]:
def square_function(time, wavelength, amplitude, phase, function_args=None):
    return amplitude * sign(sin((2*pi*time - phase)/wavelength))

In [0]:
def triangle_function(time, wavelength, amplitude, phase, function_args=None):
    return (2*amplitude/pi) * arcsin(sin((2*pi*time - phase)/wavelength))

In [0]:
def sawtooth_function(time, wavelength, amplitude, phase, function_args=None):
    return (2*amplitude/pi) * arctan(tan((2*pi*time - phase)/(2*wavelength)))

In [0]:
def pulse_function(time, wavelength, amplitude, phase, function_args):
    pulse_width = function_args['pulse_width']
    return amplitude * sign(sin((2*pi*time - phase)/wavelength)-(1-pulse_width))

In [0]:
def white_noise_function(time, wavelength, amplitude, phase, function_args):
    return random.random(len(time))*amplitude

In [0]:
def linear_asdr_envelope(sound, attack, decay, sustain, release):
    peak = sound.max()
    a = linspace(0,1,RATE*attack)
    d = linspace(1,sustain,RATE*decay)
    s = linspace(sustain,sustain,len(sound)-(RATE*(attack+decay+release)))
    r = linspace(sustain,0,RATE*release)
    envelope = append(append(a,d),append(s,r))
    return multiply(envelope,sound).astype(int16)

In [0]:
def time_warp(sound, coefficient):
    if coefficient >= 1:
        return repeat(sound, coefficient)
    else:
        slicer = int(1 / coefficient)
        return sound[0::slicer].astype(int16)

In [0]:
def lfo(sound, frequency, amount=1, wave_function=sine_function, wave_function_args={}):
    lfo = wave(function=wave_function, frequency=frequency, length=int(len(sound)/RATE), function_args=wave_function_args)
    lfo = add(lfo, lfo.max()) # shift lfo to positive
    lfo = divide(lfo,lfo.max()) # Make lfo range 0 to 1
    lfo = multiply(lfo,amount) # scale lfo by amount
    lfo = add(lfo, (1-amount)) # shift lfo up so peak is at 1 and min is (1-amount)
    return multiply(lfo,sound).astype(int16)

In [0]:
def arpeggiator(sound, step_size, frequency):
    return lfo(sound, frequency=frequency, wave_function=pulse_function, wave_function_args={'pulse_width':step_size})

In [0]:
def distortion(sound, amount):
    noise = subtract(random.random(len(sound)),0.5)*(amount*sound.max())
    return add(sound, noise).astype(int16)

In [0]:
stream.write(distortion(linear_asdr_envelope(wave(function=sawtooth_function,length=9), 1, 5, 0.5, 0), 1))

In [0]:
$ ipython locate

SyntaxError: invalid syntax (<ipython-input-996-ec7fcd2d35a7>, line 1)

#### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### TODO
- Oscilator (wav combiners)
- LFO (works on sounds but not samples?)
- ADSR envelope
- Filters (FFT?)
- Arp (very simplistic right now)
- Sampler
- How to change BPM?
    - should it be during playback like changing the stream rate
    - can I double every value in the array to make twice as long?
- distortion
- limiter
- reverb
- delay
- "utility"
- style transfer of samples (vocoder?)

### HOW TO USE
stream.write(wave(function=sine_function))

stream.write(wave(function=square_function))

stream.write(wave(function=triangle_function))

stream.write(wave(function=sawtooth_function))

stream.write(wave(function=pulse_function, function_args={'pulse_width':0.25}))

stream.write(wave(function=white_noise_function))

stream.write(linear_asdr_envelope(wave(function=square_function, length=6), 1, 1, 0.5, 1))

stream.write(wave(function=sine_function, length=3, frequency=Note.C3.value, amplitude=5000)+wave(function=sine_function, length=3, frequency=Note.C3.value*2, amplitude=2000))

stream.write(lfo(wave(function=sawtooth_function,length=9),frequency=3, amount=1))

### Sample
path = "D:\\Pat\\projects\\programming\\Accent Removal\\Samples\\wavs\\samplewav.wav"

stream.write(wavfile.read(path)[1])